# Same Class Different Size (SCDS) Comparison

This notebook compares CVCL and CLIP models on prototype evaluation where distractors are the same class but differ in size.
For example, testing a large apple against small apple, medium apple, tiny apple.

In [1]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------- ------------ 8.9/12.8 MB 69.2 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 72.9 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import os
import sys
import random
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import defaultdict

# Path setup
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir, os.pardir))

# Add discover-hidden-visual-concepts to path
DISCOVER_ROOT = os.path.join(REPO_ROOT, 'discover-hidden-visual-concepts')
sys.path.insert(0, DISCOVER_ROOT)
sys.path.insert(0, REPO_ROOT)

# Import from discover-hidden-visual-concepts repo
sys.path.append(os.path.join(DISCOVER_ROOT, 'src'))
from utils.model_loader import load_model
from models.feature_extractor import FeatureExtractor
from models.multimodal.multimodal_lit import MultiModalLitModel

# hard-coded paths
CSV_PATH = os.path.join(REPO_ROOT, 'data', 'KonkLab', 'testdata.csv')
IMG_DIR = os.path.join(REPO_ROOT, 'data', 'KonkLab', '17-objects')
MASTER_CSV = os.path.join(REPO_ROOT, 'PatrickProject', 'Chart_Generation', 'all_prototype_results.csv')

c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\clip\clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


In [3]:
# Shared Dataset and Helper Functions
class SizeImageDataset(Dataset):
    """Dataset returning (img_tensor, class, size, idx)."""
    def __init__(self, csv_path, img_dir, transform):
        self.df = pd.read_csv(csv_path)
        required = ['Filename','Class','Size']
        assert all(c in self.df for c in required), f"CSV must contain columns: {required}"
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        cls, sz = row['Class'], row['Size']
        fn = row['Filename']
        path = os.path.join(self.img_dir, cls, fn)
        img = Image.open(path).convert('RGB')
        return self.transform(img), cls, sz, idx

def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    classes = [b[1] for b in batch]
    sizes = [b[2] for b in batch]
    idxs = [b[3] for b in batch]
    return imgs, classes, sizes, idxs

def run_scds_test(model_name, seed=0, device='cuda' if torch.cuda.is_available() else 'cpu', batch_size=64, trials_per_tuple=10, max_images=None):
    """Run Same Class Different Size (SCDS) evaluation.
    
    Tests if model can identify objects when distractors are from the SAME class
    but differ in size:
    - Same class
    - Different size
    
    This is a harder test as distractors share the class but differ only in size.
    Example: large apple vs. small apple, medium apple, tiny apple
    """
    random.seed(seed)
    torch.manual_seed(seed)

    # 1) load model & transform
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    print(f"[INFO] Loaded model '{model_name}'")

    # 2) prepare DataLoader & extract embeddings
    ds = SizeImageDataset(CSV_PATH, IMG_DIR, transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)

    all_embs, all_classes, all_sizes, all_idxs = [], [], [], []
    with torch.no_grad():
        for imgs, classes, sizes, idxs in loader:
            feats = extractor.get_img_feature(imgs.to(device))
            feats = extractor.norm_features(feats).cpu()
            feats = feats.float()
            all_embs.append(feats)
            all_classes.extend(classes)
            all_sizes.extend(sizes)
            all_idxs.extend(idxs)
    all_embs = torch.cat(all_embs, dim=0)
    print(f"[INFO] Extracted embeddings for {len(all_idxs)} images")

    # 3) group by (class, size) pair
    cs_idxs = defaultdict(lambda: defaultdict(list))
    for idx, cls, sz in zip(all_idxs, all_classes, all_sizes):
        cs_idxs[cls][sz].append(idx)

    # 4) run evaluation
    total_correct = 0
    total_trials = 0
    tuple_results = {}
    
    print("[INFO] Running 4-way trials: distractors with same class but different size")
    for cls, size_groups in cs_idxs.items():
        for size, idx_list in size_groups.items():
            # Get pool: same class but different size
            other_idxs = [
                i for sz2, lst2 in size_groups.items() if sz2 != size
                for i in lst2
            ]
            
            if len(idx_list) < 1 or len(other_idxs) < 3:
                continue

            correct = 0
            for _ in range(trials_per_tuple):
                q = random.choice(idx_list)
                same_rest = [i for i in idx_list if i != q]
                if same_rest:
                    proto = all_embs[[all_idxs.index(i) for i in same_rest]].mean(0)
                else:
                    proto = all_embs[all_idxs.index(q)]
                proto = proto / proto.norm()

                distractors = random.sample(other_idxs, 3)
                candidates = [q] + distractors
                feats_cand = all_embs[[all_idxs.index(i) for i in candidates]]
                sims = feats_cand @ proto
                guess = candidates[sims.argmax().item()]

                correct += int(guess == q)
                total_correct += int(guess == q)
                total_trials += 1

            acc = correct / trials_per_tuple
            key = f"{cls}-{size}"
            tuple_results[key] = {'correct': correct, 'trials': trials_per_tuple, 'accuracy': acc}
            print(f"{cls:12s} / {size:6s} : {correct}/{trials_per_tuple} ({acc:.1%})")

    overall_acc = total_correct / total_trials if total_trials else 0.0
    print(f"\n[OK] Overall accuracy: {total_correct}/{total_trials} ({overall_acc:.1%})")
    
    # 5) save results
    summary_df = pd.DataFrame([{'Model': model_name, 'Test': 'Same-Class-Different-Size', 'Correct': total_correct, 'Trials': total_trials, 'Accuracy': overall_acc}])
    
    os.makedirs(os.path.dirname(MASTER_CSV), exist_ok=True)
    if os.path.exists(MASTER_CSV):
        summary_df.to_csv(MASTER_CSV, mode='a', header=False, index=False, float_format='%.4f')
    else:
        summary_df.to_csv(MASTER_CSV, index=False, float_format='%.4f')

    return tuple_results, overall_acc

## CVCL Test

In [4]:
# Run CVCL evaluation
cvcl_results, cvcl_overall = run_scds_test('cvcl-resnext')

print("\nCVCL Results by Class-Size:")
for key, res in cvcl_results.items():
    print(f"{key:25s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCVCL Overall Accuracy: {cvcl_overall:.1%}")

Loading checkpoint from C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt


Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt`


[INFO] Loaded model 'cvcl-resnext'
[INFO] Extracted embeddings for 1005 images
[INFO] Running 4-way trials: distractors with same class but different size
muffins      / Medium : 1/10 (10.0%)
muffins      / Large  : 6/10 (60.0%)
pitcher      / Large  : 10/10 (100.0%)
tennisracquet / Medium : 5/10 (50.0%)
phone        / Large  : 10/10 (100.0%)
phone        / Small  : 10/10 (100.0%)
headband     / Medium : 4/10 (40.0%)
headband     / Small  : 9/10 (90.0%)
bagel        / Large  : 10/10 (100.0%)
grill        / Small  : 10/10 (100.0%)
basket       / Small  : 4/10 (40.0%)
bell         / Medium : 0/10 (0.0%)
microwave    / Large  : 10/10 (100.0%)
trophy       / Small  : 4/10 (40.0%)
trophy       / Medium : 2/10 (20.0%)
fan          / Medium : 9/10 (90.0%)
fan          / Small  : 10/10 (100.0%)
fan          / Large  : 10/10 (100.0%)
lei          / Medium : 2/10 (20.0%)
lei          / Small  : 3/10 (30.0%)
stapler      / Medium : 2/10 (20.0%)
stapler      / Small  : 6/10 (60.0%)
exercise_equipm

## CLIP Test

In [5]:
# Run CLIP evaluation
clip_results, clip_overall = run_scds_test('clip-resnext')

print("\nCLIP Results by Class-Size:")
for key, res in clip_results.items():
    print(f"{key:25s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCLIP Overall Accuracy: {clip_overall:.1%}")

[INFO] Loaded model 'clip-resnext'


c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


[INFO] Extracted embeddings for 1005 images
[INFO] Running 4-way trials: distractors with same class but different size
muffins      / Medium : 3/10 (30.0%)
muffins      / Large  : 2/10 (20.0%)
pitcher      / Large  : 10/10 (100.0%)
tennisracquet / Medium : 4/10 (40.0%)
phone        / Large  : 10/10 (100.0%)
phone        / Small  : 10/10 (100.0%)
headband     / Medium : 9/10 (90.0%)
headband     / Small  : 8/10 (80.0%)
bagel        / Large  : 10/10 (100.0%)
grill        / Small  : 10/10 (100.0%)
basket       / Small  : 0/10 (0.0%)
bell         / Medium : 0/10 (0.0%)
microwave    / Large  : 10/10 (100.0%)
trophy       / Small  : 1/10 (10.0%)
trophy       / Medium : 4/10 (40.0%)
fan          / Medium : 7/10 (70.0%)
fan          / Small  : 8/10 (80.0%)
fan          / Large  : 10/10 (100.0%)
lei          / Medium : 2/10 (20.0%)
lei          / Small  : 0/10 (0.0%)
stapler      / Medium : 4/10 (40.0%)
stapler      / Small  : 3/10 (30.0%)
exercise_equipment / Medium : 0/10 (0.0%)
handgun     